In [19]:
import tensorflow as tf
import os
import pandas as pd
import json
import ast
import json
from datasets import Dataset
from transformers import CamembertTokenizerFast, DataCollatorForTokenClassification, CamembertForTokenClassification, TrainingArguments, Trainer
import numpy as np
from seqeval.metrics import classification_report
import torch
from torchsummary import summary

os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [21]:
def load_json_safely(file_path):
    try:
        
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except:
        try:
            
            data = []
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    try:
                        
                        data.append(json.loads(line.strip()))
                    except:
                        continue
            return data
        except:
            
            return pd.read_json(file_path, lines=True).to_dict('records')


train_data = load_json_safely('./annotations/train_extended_bio_feb.json')
test_data = load_json_safely('./annotations/val_extended_bio_feb.json')

len(train_data), len(test_data)

(59900, 14758)

In [ ]:
train_dataset = Dataset.from_dict({"tokens": [item["tokens"] for item in train_data],
                                   "ner_tags": [item["tags"] for item in train_data]})
test_dataset = Dataset.from_dict({"tokens": [item["tokens"] for item in test_data],
                                  "ner_tags": [item["tags"] for item in test_data]})


tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")
data_collator = DataCollatorForTokenClassification(tokenizer)


label_list = ["O", "B-country", "I-country", "B-region", "I-region", "B-departement", "I-departement", "B-province", "I-province", "B-village", "I-village"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

def convert_tags_to_ids(examples):
    examples["ner_tags"] = [[label2id[tag] for tag in tags] for tags in examples["ner_tags"]]
    return examples


train_dataset = train_dataset.map(convert_tags_to_ids, batched=True)
test_dataset = test_dataset.map(convert_tags_to_ids, batched=True)


def tokenize_and_align_labels(examples):

    tokenized = tokenizer(
        examples["tokens"],
        truncation=True,
        padding=True,
        is_split_into_words=True,
        return_offsets_mapping=True
    )

    batch_labels  = []
    batch_offsets = []


    for i, labels in enumerate(examples["ner_tags"]):
        word_ids      = tokenized.word_ids(batch_index=i)
        offsets_i     = tokenized["offset_mapping"][i]
        label_ids, offs = [], []

        prev_word_idx = None
        for idx, word_idx in enumerate(word_ids):
            if word_idx is None:

                label_ids.append(-100)
                offs.append((None, None))
            elif word_idx != prev_word_idx:

                label_ids.append(labels[word_idx])
                offs.append(offsets_i[idx])
            else:

                label_ids.append(-100)
                offs.append((None, None))
            prev_word_idx = word_idx

        batch_labels.append(label_ids)
        batch_offsets.append(offs)


    tokenized["labels"]       = batch_labels
    tokenized["char_offsets"] = batch_offsets



    return tokenized


train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 14758/14758 [00:19<00:00, 766.69 examples/s]


In [23]:
model = CamembertForTokenClassification.from_pretrained("camembert-base", num_labels=len(label_list))

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model.to(torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu'))

CamembertForTokenClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=Tr

In [27]:
# Geler les embeddings
for param in model.roberta.embeddings.parameters():
    param.requires_grad = False

# #Geler les premières couches du CamembertEncoder (par exemple, les 8 premières sur 12)
# for layer in model.roberta.encoder.layer[:-4]:  # Ne pas entraîner les 8 premières couches
#     for param in layer.parameters():
#         param.requires_grad = False

In [28]:
# Compter les paramètres entraînables vs gelés
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
frozen_params = total_params - trainable_params

print(f"Total de paramètres: {total_params:,}")
print(f"Paramètres entraînables: {trainable_params:,}")
print(f"Paramètres gelés: {frozen_params:,}")
print(f"Pourcentage entraînable: {100 * trainable_params / total_params:.2f}%")

# Détails par composant
print("\n--- Détails par composant ---")
for name, module in model.named_children():
    module_params = sum(p.numel() for p in module.parameters())
    module_trainable = sum(p.numel() for p in module.parameters() if p.requires_grad)
    print(f"{name}: {module_trainable:,}/{module_params:,} paramètres entraînables")


Total de paramètres: 110,039,819
Paramètres entraînables: 85,062,923
Paramètres gelés: 24,976,896
Pourcentage entraînable: 77.30%

--- Détails par composant ---
roberta: 85,054,464/110,031,360 paramètres entraînables
dropout: 0/0 paramètres entraînables
classifier: 8,459/8,459 paramètres entraînables


In [29]:
# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="/data/charles/agile/camembert-ner-finetuned",
    evaluation_strategy="steps",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=70,
    weight_decay=0.01,
    save_strategy="steps",
    save_steps = 100,
    #
    do_train=True,
    do_predict=True,
    save_total_limit=100,
    push_to_hub=False,
    #metric_for_best_model="eval_loss",
    #load_best_model_at_end=True
)



# Fonction pour calculer les métriques
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [[id2label[p] for (p, l) in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)]

    results = classification_report(true_labels, true_predictions, output_dict=True)
    return {
        "precision": results["micro avg"]["precision"],
        "recall": results["micro avg"]["recall"],
        "f1": results["micro avg"]["f1-score"],
    }

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/charles/stkgfs/stkgfs/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Precision,Recall,F1
500,0.245600,0.055543,0.254341,0.156014,0.193398
1000,0.030400,0.014886,0.867308,0.837670,0.852231
1500,0.012400,0.008987,0.888305,0.916525,0.902195
2000,0.007300,0.005082,0.938660,0.941978,0.940316
2015,0.007300,0.004939,0.934590,0.950933,0.942691


/home/charles/stkgfs/stkgfs/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyboardInterrupt: 

In [30]:
trainer.evaluate()

{'eval_loss': 0.004938560537993908,
 'eval_precision': 0.9345902626580812,
 'eval_recall': 0.950933257918552,
 'eval_f1': 0.9426909328785364}

In [33]:
import json
from datasets import Dataset
from transformers import CamembertTokenizerFast, DataCollatorForTokenClassification, CamembertForTokenClassification, Trainer
import numpy as np
from seqeval.metrics import classification_report

# Fonction pour charger les données JSON
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Charger les données de test
test_data = load_json('./annotations/test_extended_bio_feb.json')

# Convertir en format Dataset
test_dataset = Dataset.from_dict({
    "tokens": [item["tokens"] for item in test_data],
    "ner_tags": [item["tags"] for item in test_data]
})

# Initialiser le tokenizer
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")

# Définir la liste des étiquettes et créer les mappings label <-> id
label_list = ["O", "B-country", "I-country", "B-region", "I-region", "B-departement", "I-departement", "B-province", "I-province", "B-village", "I-village"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

# Fonction pour convertir les étiquettes en IDs
def convert_tags_to_ids(examples):
    examples["ner_tags"] = [[label2id[tag] for tag in tags] for tags in examples["ner_tags"]]
    return examples

# Appliquer la conversion sur le dataset de test
test_dataset = test_dataset.map(convert_tags_to_ids, batched=True)

# Fonction pour tokeniser les entrées et aligner les étiquettes
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding=True,
        is_split_into_words=True,
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignorer les tokens spéciaux
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])  # Nouveau mot
            else:
                label_ids.append(-100)  # Même mot, ignorer
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Appliquer la tokenisation et l'alignement des étiquettes
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

# Charger le modèle fine-tuné
model = CamembertForTokenClassification.from_pretrained("/data/charles/agile/camembert-ner-finetuned/checkpoint-2000", num_labels=len(label_list))

# Initialiser le Trainer sans entraînement, uniquement pour l'évaluation
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
)

# Effectuer les prédictions sur le jeu de test
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=2)

# Convertir les IDs en étiquettes
true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
true_predictions = [[id2label[p] for (p, l) in zip(prediction, label) if l != -100]
                    for prediction, label in zip(predictions, labels)]

Map: 100%|██████████| 11594/11594 [00:06<00:00, 1800.65 examples/s]


In [34]:
# Afficher le rapport de classification
print(classification_report(true_labels, true_predictions))

              precision    recall  f1-score   support

     country       0.97      0.99      0.98      4648
 departement       0.95      0.96      0.95      6744
    province       0.91      0.92      0.91       541
      region       0.98      0.99      0.99      1433
     village       0.84      0.81      0.82      3236

   micro avg       0.94      0.94      0.94     16602
   macro avg       0.93      0.93      0.93     16602
weighted avg       0.94      0.94      0.94     16602

